In [5]:
!pip install mesa==2.4.0  

In [6]:
import mesa
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.colors import ListedColormap
import numpy as np
import time
import mesa
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
from IPython.display import display, clear_output
import ipywidgets as widgets
import asyncio
import threading  
import nest_asyncio
import warnings
from matplotlib.patches import Patch

warnings.filterwarnings('ignore')
nest_asyncio.apply()

# Creating a the Agent-Based Model
The Agent-based model is performed on a grid. Each cell of the grid can have various properties and components of the model. Once the model is completely setup then we can run the model. Running the model consists of individual time steps in which the model, agents, and other components perform some tasks. We will begin with creating the agents of the model.

## Creating Agents
Mesa uses agents as the main object of modelling. We will create an agent that will represent a section of the forest. Agents are a class in Mesa that we will extend and pass it the required variables of the model and the agent's unique id. Agents also have a method called step that we need to overload. During the step the agent will determine if it is on fire and if it is, then it will light its neighbors on fire.

In [ ]:
class Forest(mesa.Agent):
    def __init__(self, unique_id, model, position):
        super().__init__(unique_id, model)
        self.pos = position
        self.status = "Fine"
        
    def step(self):
        if self.status == "Burning":
            for neighbor in self.model.grid.iter_neighbors(self.pos, True):
                if neighbor.status == "Fine":
                    if self.model.random.random() > 0.25:
                        neighbor.status = "Burning"
                self.status = "Burned"
        
        #TODO
        #Develop logic for regenerating the forest after being burnt

## Creating the Model
The model is a Mesa class that we will extend just like we did with the agents. The model is the controller of the overall Agent-based Model

In [7]:
class ForestFire(mesa.Model):
    def __init__(
        self,
        width = 100,
        height = 100,
        density=0.65):
        
        super().__init__()
        self.width = width
        self.height = height
        self.density = density
        self.schedule = mesa.time.RandomActivation(self)
        self.grid = mesa.space.SingleGrid(self.width, self.height, torus=False)
        self.datacollector = mesa.DataCollector(
            model_reporters={
                "Fine": lambda model: self.count_type(model, "Fine"),
                "Burning": lambda model: self.count_type(model, "Burning"),
                "Burned": lambda model: self.count_type(model, "Burned"),
                #TODO: Add output for regeneration
            }
        )
        self._initialize_forest()
        self.datacollector.collect(self)
        
        
    def _initialize_forest(self):
        for _contents, pos in self.grid.coord_iter():
            forest = Forest(self.next_id(), self, pos)
            if self.random.random() < self.density:
                if pos[0] == 0: #set this section to burning
                    forest.status = "Burning"
                else:
                    forest.status = "Fine"
            else:
                forest.status = "Burned"
            self.schedule.add(forest)
            self.grid.place_agent(forest, pos)
            
    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)
        
    @staticmethod
    def count_type(model, status):
        count = 0
        for forest in model.schedule.agents:
            if forest.status == status:
                count += 1
        return count
                    

## Create an Interactive Form
Here we will create an interactive form in the Notebook so we can experiment with our model.

In [8]:
class InteractiveForestFire:
    def __init__(self):
        self.model = None
        self.running = False
        self.step_count = 0

        # Runners & guards
        self._task = None              
        self._thread = None            
        self._stop_event = threading.Event()
        self._rendering = False        # prevent overlapping redraws

        # Create widgets
        self.play_button  = widgets.Button(description="▶️ Play",  button_style='success')
        self.step_button  = widgets.Button(description="⏭️ Step")
        self.reset_button = widgets.Button(description="🔄 Reset", button_style='danger')
        self.stop_button  = widgets.Button(description="⏹️ Stop",  button_style='info')

        self.width_slider  = widgets.IntSlider(value=100, min=20, max=150, step=10, description='Width:')
        self.height_slider = widgets.IntSlider(value=100, min=20, max=150, step=10, description='Height:')
        self.density_slider = widgets.FloatSlider(value=0.65, min=0.1, max=1.0, step=0.05, description='Density:')
        

        self.output = widgets.Output()

        # Connect buttons
        self.play_button.on_click(self.toggle_play)
        self.step_button.on_click(self.do_step)
        self.reset_button.on_click(self.reset_simulation)
        self.stop_button.on_click(self.stop_simulation)

        # Initialize model
        self.reset_simulation()

    # ---------- UI helpers ----------
    def _set_play_ui(self, playing: bool):
        if playing:
            self.play_button.description = "⏸️ Pause"
            self.play_button.button_style = 'warning'
        else:
            self.play_button.description = "▶️ Play"
            self.play_button.button_style = 'success'

    # ---------- Start/Stop/reset ----------
    def toggle_play(self, button):
        self.running = not self.running
        if self.running:
            self._set_play_ui(True)
            self.run_simulation()
        else:
            self.stop_simulation()

    def stop_simulation(self, button=None):
        # Stop flag
        self.running = False
        self._stop_event.set()

        # Cancel asyncio task if any
        if self._task is not None:
            t = self._task
            self._task = None
            if not t.done():
                t.cancel()

        # Join thread if any
        if self._thread is not None and self._thread.is_alive():
            self._thread.join(timeout=1.0)
        self._thread = None

        self._set_play_ui(False)
        # Optional clean redraw
        self.update_display()

    def reset_simulation(self, button=None):
        # Fully stop any runner
        self.stop_simulation()

        # Reset state
        self.step_count = 0

        # Rebuild model
        self.model = ForestFire(
            width=self.width_slider.value,
            height=self.height_slider.value,
            density=self.density_slider.value,
            # regeneration_time=self.regen_time_slider.value,
            # regeneration_rate=self.regen_rate_slider.value
        )
        self.update_display()

    # ---------- Runner selection ----------
    def run_simulation(self):
        if not self.running:
            return

        # If something is already running, don’t start another
        if (self._task is not None and not self._task.done()) or \
           (self._thread is not None and self._thread.is_alive()):
            return

        # Clear stop signal for new run
        self._stop_event.clear()

        # Prefer asyncio in environments where a loop is running
        try:
            loop = asyncio.get_running_loop()
            # If we got here without raising, schedule the async loop
            self._task = loop.create_task(self._async_loop())
        except RuntimeError:
            # No running loop (classic notebook/Colab older setups) -> use thread
            self._thread = threading.Thread(target=self._thread_loop, daemon=True)
            self._thread.start()

    async def _async_loop(self):
        try:
            while self.running:
                self.do_step()
                await asyncio.sleep(0.1)  # yield to UI
        except asyncio.CancelledError:
            pass

    def _thread_loop(self):
        while self.running and not self._stop_event.is_set():
            self.do_step()
            time.sleep(0.1)

    # ---------- One step + render ----------
    def do_step(self, button=None):
        if not self.model:
            return
        if self._rendering:
            return  # skip if a draw is underway
        self._rendering = True
        try:
            self.model.step()
            self.step_count += 1
            self.update_display()
        finally:
            self._rendering = False

    # ---------- Drawing ----------
    def update_display(self):
        if self.model is None:
            return

        with self.output:
            clear_output(wait=True)

            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

            # Grid visualization
            W, H = self.model.grid.width, self.model.grid.height
            grid = np.zeros((W, H))
            status_to_num = {"Burned": 0, "Fine": 1, "Burning": 2}

            for agent in self.model.agents:
                if hasattr(agent, 'pos') and hasattr(agent, 'status'):
                    x, y = agent.pos
                    if 0 <= x < W and 0 <= y < H:
                        grid[x][y] = status_to_num.get(agent.status, 0)

            cmap = ListedColormap(["#3D2B1F", "#00AA00", "#FF0000"])
            ax1.imshow(grid.T, cmap=cmap, origin="lower", vmin=0, vmax=2)
            ax1.set_title(f"Forest Grid - Step {self.step_count}", fontsize=14, fontweight='bold')
            ax1.set_xlabel("X")
            ax1.set_ylabel("Y")
            ax1.set_xticks([])
            ax1.set_yticks([])
            legend_elements = [
                Patch(facecolor='#3D2B1F', label='Burned'),
                Patch(facecolor='#00AA00', label='Fine'),
                Patch(facecolor='#FF0000', label='Burning')
            ]
            ax1.legend(handles=legend_elements, loc='upper right')

            # Chart visualization
            df = self.model.datacollector.get_model_vars_dataframe()
            if df is not None and not df.empty:
                ax2.plot(df.index, df["Fine"],    label="Fine Trees", linewidth=2)
                ax2.plot(df.index, df["Burning"], label="Burning",    linewidth=2)
                ax2.plot(df.index, df["Burned"],  label="Burned",     linewidth=2)

                ax2.set_xlabel("Step", fontsize=12)
                ax2.set_ylabel("Count", fontsize=12)
                ax2.set_title("Forest Status Over Time", fontsize=14, fontweight='bold')
                ax2.legend(fontsize=10)
                ax2.grid(True, alpha=0.3)
                ax2.set_ylim(0, W * H)

            plt.tight_layout()
            plt.show()

    # ---------- UI layout ----------
    def display(self):
        controls = widgets.HBox([self.play_button, self.step_button, self.reset_button, self.stop_button])
        sliders = widgets.VBox([
            self.width_slider,
            self.height_slider,
            self.density_slider,
            # self.regen_time_slider,
            # self.regen_rate_slider
        ])

        ui = widgets.VBox([
            widgets.HTML("<h1>🔥 Forest Fire Simulation with Regeneration</h1>"),
            controls,
            sliders,
            self.output
        ])

        display(ui)

# Run the simulation
sim = InteractiveForestFire()
sim.display()

# Power of Experimentation

The power of Agent-based Modeling is that you can easily explore many hypotheses and determine how they effective they are by adding them to the model and running it. 

Using the model and agent definitions above, you can explore many other scenarios by which forest fire damage can be mitigated. Try adding a status of "Fire Break" that can not catch fire and see how that benefits or hurts the forests during fire conditions. 
